In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
import datetime as dt
import scipy.signal as ss

In [11]:
from sklearn.preprocessing import OneHotEncoder

In [2]:
holidays_events = pd.read_csv("https://www.dropbox.com/s/bxyamlpevkiwwoq/holidays_events.csv?dl=1")
oil = pd.read_csv("https://www.dropbox.com/s/l6ln0ztl4m0pw3a/oil.csv?dl=1",parse_dates=['date'],index_col='date')
sample_submission = pd.read_csv("https://www.dropbox.com/s/68jjl61x6u3klos/sample_submission.csv?dl=1")
stores = pd.read_csv("https://www.dropbox.com/s/lcxn6r9bs2exguq/stores.csv?dl=1")
test = pd.read_csv("https://www.dropbox.com/s/cvdo1gn7r5lu2uz/test.csv?dl=1",index_col='id')
train = pd.read_csv("https://www.dropbox.com/s/s8p2b5awnuqfk0d/train.csv?dl=1",index_col='id')
transactions = pd.read_csv("https://www.dropbox.com/s/92fij9bcwt0e0cj/transactions.csv?dl=1")

Dzielimy próbkę train na treningową i testową.

In [3]:
train2 = train.loc[(train['date']<'2016-06-01')]
test2 = train.loc[(train['date']>='2016-06-01')]
oil_train2 = oil.loc[(oil.index<'2016-06-01')].fillna(method ="bfill")
oil_test2 = oil.loc[(oil.index>='2016-06-01')].fillna(method ="bfill")

Dołączamy do zbioru train2 informacje o świętach

In [4]:
train2_merged = train2.merge(holidays_events,how="left",left_on=['date'],right_on=['date'])

In [7]:
train2_merged.head()

,date,store_nbr,family,sales,onpromotion,type,locale,locale_name,description,transferred
2252443,2016-05-31,9,POULTRY,493.638980,0,NaN,NaN,NaN,NaN,NaN
2252444,2016-05-31,9,PREPARED FOODS,89.048996,1,NaN,NaN,NaN,NaN,NaN
2252445,2016-05-31,9,PRODUCE,2367.612000,138,NaN,NaN,NaN,NaN,NaN
2252446,2016-05-31,9,SCHOOL AND OFFICE SUPPLIES,4.000000,0,NaN,NaN,NaN,NaN,NaN
2252447,2016-05-31,9,SEAFOOD,19.400000,1,NaN,NaN,NaN,NaN,NaN


In [36]:
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'team' column 
encoder_df = pd.DataFrame(encoder.fit_transform(train2_merged[['locale']]).toarray())

#merge one-hot encoded columns back with original DataFrame
final_df = train2_merged.join(encoder_df)

#drop 'team' column
final_df.drop('locale', axis=1, inplace=True)

#rename columns
#final_df = final_df.rename(columns={'0':'Local','1':'National','2':'Regional','3':'NA'})
final_df.columns = ['date', 'store_nbr', 'family', 'sales','onpromotion','type','local_name','description','transferred','isLocal','isNational','isRegional','isNA']



In [37]:
final_df.head()

,date,store_nbr,family,sales,onpromotion,type,local_name,description,transferred,isLocal,isNational,isRegional,isNA
0,2013-01-01,1,AUTOMOTIVE,0.0,0,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0
1,2013-01-01,1,BABY CARE,0.0,0,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0
2,2013-01-01,1,BEAUTY,0.0,0,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0
3,2013-01-01,1,BEVERAGES,0.0,0,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0
4,2013-01-01,1,BOOKS,0.0,0,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0


In [38]:
final_df['dayofweek'] = pd.DatetimeIndex(final_df['date']).dayofweek + 1

In [39]:
final_df.head()

,date,store_nbr,family,sales,onpromotion,type,local_name,description,transferred,isLocal,isNational,isRegional,isNA,dayofweek
0,2013-01-01,1,AUTOMOTIVE,0.0,0,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0,2
1,2013-01-01,1,BABY CARE,0.0,0,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0,2
2,2013-01-01,1,BEAUTY,0.0,0,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0,2
3,2013-01-01,1,BEVERAGES,0.0,0,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0,2
4,2013-01-01,1,BOOKS,0.0,0,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0,2


Wybieramy obserwacje dla family=AUTOMOTIVE

In [44]:
train_automotive = final_df.loc[(final_df['family']=='AUTOMOTIVE')]

In [60]:
train_automotive.head(100)

,date,store_nbr,family,sales,onpromotion,type,local_name,description,transferred,isLocal,isNational,isRegional,isNA,dayofweek
0,2013-01-01,1,AUTOMOTIVE,0.0,0,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0,2
33,2013-01-01,10,AUTOMOTIVE,0.0,0,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0,2
66,2013-01-01,11,AUTOMOTIVE,0.0,0,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0,2
99,2013-01-01,12,AUTOMOTIVE,0.0,0,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0,2
132,2013-01-01,13,AUTOMOTIVE,0.0,0,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3135,2013-01-02,47,AUTOMOTIVE,11.0,0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,3
3168,2013-01-02,48,AUTOMOTIVE,5.0,0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,3
3201,2013-01-02,49,AUTOMOTIVE,5.0,0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,3
3234,2013-01-02,5,AUTOMOTIVE,6.0,0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,3


Wyliczamy średnią sprzedaż na daną datę

In [66]:
train_automotive2= train_automotive.groupby(['date'])['sales'].mean().to_frame()

In [65]:
train_automotive2.head()

,sales
date,
2013-01-01,0.000000
2013-01-02,4.722222
2013-01-03,2.981481
2013-01-04,3.129630
2013-01-05,6.333333


In [107]:
train_automotive_merged = train_automotive2.merge(holidays_events,how="left",left_on=['date'],right_on=['date'])

encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'team' column 
encoder_df = pd.DataFrame(encoder.fit_transform(train_automotive_merged[['locale']]).toarray())

#merge one-hot encoded columns back with original DataFrame
final_train_automotive = train_automotive_merged.join(encoder_df)

#drop 'team' column
final_train_automotive.drop('locale', axis=1, inplace=True)

#rename columns
#final_df = final_df.rename(columns={'0':'Local','1':'National','2':'Regional','3':'NA'})
final_train_automotive.columns = ['date', 'sales','type','local_name','description','transferred','isLocal','isNational','isRegional','isNA']


In [72]:
final_train_automotive.head()

,date,sales,type,local_name,description,transferred,isLocal,isNational,isRegional,isNA
0,2013-01-01,0.000000,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0
1,2013-01-02,4.722222,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0
2,2013-01-03,2.981481,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0
3,2013-01-04,3.129630,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0
4,2013-01-05,6.333333,Work Day,Ecuador,Recupero puente Navidad,False,0.0,1.0,0.0,0.0


In [108]:
final_train_automotive['dayofweek'] = pd.DatetimeIndex(final_train_automotive['date']).dayofweek + 1

In [109]:
final_train_automotive.head()

,date,sales,type,local_name,description,transferred,isLocal,isNational,isRegional,isNA,dayofweek
0,2013-01-01,0.000000,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0,2
1,2013-01-02,4.722222,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,3
2,2013-01-03,2.981481,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,4
3,2013-01-04,3.129630,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,5
4,2013-01-05,6.333333,Work Day,Ecuador,Recupero puente Navidad,False,0.0,1.0,0.0,0.0,6


In [110]:
final_train_automotive.drop(["description","date","type","local_name","transferred"],axis=1,inplace=True)

In [111]:
final_train_automotive.head()

,sales,isLocal,isNational,isRegional,isNA,dayofweek
0,0.000000,0.0,1.0,0.0,0.0,2
1,4.722222,0.0,0.0,0.0,1.0,3
2,2.981481,0.0,0.0,0.0,1.0,4
3,3.129630,0.0,0.0,0.0,1.0,5
4,6.333333,0.0,1.0,0.0,0.0,6


Tworzymy model liniowy

In [112]:
X = final_train_automotive.drop(['sales'],axis=1)
x = sm.add_constant(X)
Y = final_train_automotive['sales']
y = pd.DataFrame(Y)

In [105]:
x.head()

,const,type,local_name,transferred,isLocal,isNational,isRegional,isNA,dayofweek
0,1.0,Holiday,Ecuador,False,0.0,1.0,0.0,0.0,2
1,1.0,NaN,NaN,NaN,0.0,0.0,0.0,1.0,3
2,1.0,NaN,NaN,NaN,0.0,0.0,0.0,1.0,4
3,1.0,NaN,NaN,NaN,0.0,0.0,0.0,1.0,5
4,1.0,Work Day,Ecuador,False,0.0,1.0,0.0,0.0,6


In [113]:
model = sm.OLS(Y,X).fit()

In [115]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.285
Model:                            OLS   Adj. R-squared:                  0.283
Method:                 Least Squares   F-statistic:                     125.6
Date:                Wed, 20 Apr 2022   Prob (F-statistic):           2.76e-90
Time:                        20:45:53   Log-Likelihood:                -2417.1
No. Observations:                1264   AIC:                             4844.
Df Residuals:                    1259   BIC:                             4870.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
isLocal        3.7267      0.203     18.368      0.0

Spróbujmy bez liczenia średniej dla danej daty

In [116]:
train_automotive.head()

,date,store_nbr,family,sales,onpromotion,type,local_name,description,transferred,isLocal,isNational,isRegional,isNA,dayofweek
0,2013-01-01,1,AUTOMOTIVE,0.0,0,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0,2
33,2013-01-01,10,AUTOMOTIVE,0.0,0,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0,2
66,2013-01-01,11,AUTOMOTIVE,0.0,0,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0,2
99,2013-01-01,12,AUTOMOTIVE,0.0,0,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0,2
132,2013-01-01,13,AUTOMOTIVE,0.0,0,Holiday,Ecuador,Primer dia del ano,False,0.0,1.0,0.0,0.0,2


In [119]:
X = train_automotive.drop(['sales','family','type','local_name','description','transferred','date','store_nbr'],axis=1)
x = sm.add_constant(X)
Y = train_automotive['sales']
y = pd.DataFrame(Y)
model = sm.OLS(Y,X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     611.1
Date:                Wed, 20 Apr 2022   Prob (F-statistic):               0.00
Time:                        20:55:36   Log-Likelihood:            -2.2054e+05
No. Observations:               68256   AIC:                         4.411e+05
Df Residuals:                   68250   BIC:                         4.412e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
onpromotion     3.9405      0.118     33.280      